In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import creep_event_picker as cep
import obspy
from matplotlib.dates import num2date
import matplotlib.dates as mdates
%matplotlib qt
import math
import h5py

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def check_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path, exist_ok=True)

Import creepmeter database

In [3]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
#Creepmeter_dataframe = pd.read_excel('../../Data/creepmeter_lookup_and_processing_progress.xlsx',index_col=0)

In [4]:
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['picked']!='No'].index,inplace=True)
Creepmeter_dataframe.reset_index(inplace=True,drop=True)
Creepmeter_dataframe.sort_values(by='Start Time')

,Network,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Length,Depth,Obliquity,Creepmeter_full_name,Fault
117,HOL,XMR1,XMR1_10T,1980-01-30T00:00:00.000000Z,2014-03-11T20:30:00.000000Z,0.001667,10T,36.5950,-121.1870,10 m,cm,30.0,Melendy Ranch,San Andreas Fault - Hollister
108,HOL,XHR1,XHR1_10T,1981-09-15T00:00:00.000000Z,1986-12-05T23:50:00.000000Z,0.001667,10T,36.7720,-121.4220,30 m,NaN,22.0,Harris Ranch,San Andreas Fault - Hollister
115,PARK,XMM1,XMM1_10T,1984-09-26T00:00:00.000000Z,2023-04-26T07:50:00.000000Z,0.001667,10T,35.9580,-120.5020,26 m,1 m,30.0,Middle Mtn,San Andraes Fault - Parkfield
107,PARK,XGH1,XGH1_10T,1985-04-03T00:00:00.000000Z,2022-06-28T09:10:00.000000Z,0.001667,10T,35.8200,-120.3480,10 m,cm,30.0,Gold Hill,San Andraes Fault - Parkfield
121,PARK,XSC1,XSC1_10T,1985-04-12T00:00:00.000000Z,2022-11-15T00:00:00.000000Z,0.001667,10T,36.0650,-120.6280,10 m,cm,27.0,Slacks Canyon,San Andraes Fault - Parkfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,EAF,TAS1,TAS1_1T,2023-06-01T10:55:00.000000Z,2024-05-26T09:07:00.000000Z,0.016667,1T,38.2049,38.7872,52 (originally 9m),NaN,30.0,Tasmis,East Anatolian Fault
57,EAF,KAR1,KAR1_1T,2023-06-05T05:38:00.000000Z,2024-05-26T06:49:00.000000Z,0.016667,1T,38.0697,38.4983,6 m,NaN,30.0,Karakose,East Anatolian Fault
127,EAF,YZE1,YZE1_1T,2023-09-07T10:36:07.000000Z,2024-05-26T12:59:07.000000Z,0.016667,1T,38.1791,38.7526,20 m,NaN,30.0,Yazica East,East Anatolian Fault
110,HOL,XHR3,XHR3_1T,2024-02-28T23:21:00.000000Z,2024-10-09T19:27:00.000000Z,0.016667,1T,36.7720,-121.4220,30 m,NaN,22.0,Harris Ranch,San Andreas Fault - Hollister


Import data

In [5]:
creepmeter ='BAL1'

In [6]:
creepmeter_info = Creepmeter_dataframe.loc[Creepmeter_dataframe.index[Creepmeter_dataframe.Creepmeter_abbrv==creepmeter]]

In [7]:
creepmeter_info.reset_index(inplace=True,drop=True)
creepmeter_info

,Network,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Length,Depth,Obliquity,Creepmeter_full_name,Fault
0,EAF,BAL1,BAL1_1T,2023-03-12T10:47:55.000000Z,2023-09-05T13:14:55.000000Z,0.016667,1T,37.9903,38.199,5 m,NaN,30.0,Balikburnu,East Anatolian Fault


In [8]:

plt.close('all')
for i in range(len(creepmeter_info)):

    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=creepmeter_info['Creepmeter_abbrv'].iloc[i]), 'r') as f:
            # Loop through each key in the file
            keys = list(f.keys())
            for key in f.keys():
                if key not in ['EASH_1.0mins', 'EASH_10.0mins','Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                    print("Key:", key)
                    print('extracting data')
                    data = f[key]  # Access the dataset
                    data_keys = list(data.keys())
                    print(data_keys)
                    slip = data[data_keys[0]][:]
                    time =  data[data_keys[1]][:]
                    decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                    decoded_time = pd.to_datetime(decoded_time)
                    sample_rate = f[key].attrs['sampling_rate']

                    path_save = "../../Data/DATA_tidied/Picks/"
                    
                    print('Load in creepmeter data from a SAC file')
                    tm_int = decoded_time
                    creep_int = slip
                    


                    '''print('Crop to before Jan 2024 with overhang for events that start December 2023')
                    boolarr_Jan24 = tm_int<=np.datetime64('2024-02-01T00:00:00')
                    tm_int = tm_int[boolarr_Jan24]
                    creep_int = creep_int[boolarr_Jan24]'''

                    plt.figure()
                    plt.plot(tm_int,creep_int)
                    plt.show()

                    print('scaling data if from Roger')
                    '''if creepmeter_info['data source'].iloc[i] == 'Roger':
                        creep_int = creep_int/math.cos(np.deg2rad(int(creepmeter_info['Obliquity'].iloc[i])))'''
                    #creep_int = creep_int/math.cos(np.deg2rad(int(creepmeter_info['Obliquity'].iloc[i])))
                    print('Picking')
                    
                    if creepmeter == 'SIV1':
                        time_in_mins = int(2*10080/sample_rate)#10080
                    else:
                        time_in_mins = int(4*10080/sample_rate)#10080
                    try:
                        df_picks = pd.read_csv('../../Data/DATA_tidied/Picks/{q}_picks.csv'.format(q =creepmeter_info.File_code.iloc[i]))
                    except FileNotFoundError:
                        df_picks=pd.DataFrame(columns=['ST'])

                    
                    for k in range(round(len(tm_int)/time_in_mins)+1):
                        fig=plt.figure(figsize=(20,10))
                        fig.tight_layout()
                        ax = plt.subplot(1,1,1)
                        plt.title('{q}/{u}'.format(q=k+1,u=round(len(tm_int)/time_in_mins)+1))
                        try:
                            plt.plot(tm_int[k*time_in_mins:(k+1)*time_in_mins],creep_int[k*time_in_mins:(k+1)*time_in_mins]-creep_int[k*time_in_mins:(k+1)*time_in_mins][0], 
                                    marker='o', color='b', linestyle='-', linewidth=2, markersize=6)
                        except IndexError:
                            dummy=10
                        bottom,top = ax.get_ylim()
                        try:
                            for q in range(len(df_picks)):
                                plt.plot([pd.to_datetime(df_picks.ST.iloc[q]),pd.to_datetime(df_picks.ST.iloc[q])],[bottom,top],color='orange')
                        except:
                            print('woo')
                        try:
                            ax.set_xlim([tm_int[k*time_in_mins],tm_int[(k+1)*time_in_mins]])
                        except IndexError:
                            dummy=10
                        ax.set_ylim([bottom,top])
                        scale = 1.1
                        zp = cep.ZoomPan()
                        figZoom = zp.zoom_factory(ax, base_scale = scale)
                        figPan = zp.pan_factory(ax)
                        s = 1
                        plt.show()
                        pts = plt.ginput(100,timeout = -1)
                        plt.close('all')
                        for sublist in pts:

                            df = pd.DataFrame({'ST':[sublist[0]]})
                            df_picks = pd.concat([df_picks,df])
                    print(len(df_picks))
                    
                    if len(df_picks) >0:
                        for r in range(len(df_picks)):

                            try:
                                time_convert = mdates.num2date(df_picks.ST.iloc[r])
                                s3 = pd.Series(time_convert).dt.round("1min")
                                df_picks.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)   
                            except TypeError:
                                dummy=10
                        
                        df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
                        df_picks.ST = pd.to_datetime(df_picks.ST)
                        df_picks.sort_values(by='ST',inplace=True)
                        #df_picks.reset_index(inplace=True)
                        df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =creepmeter_info.File_code.iloc[i]))

                        
                        plt.close('all')
                        df_picks_end=pd.DataFrame(columns=['ET'])
                        for j in range(len(df_picks)):
                            boolarr = (tm_int>=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None)-dt.timedelta(hours=2)) \
                                & (tm_int<=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None)+dt.timedelta(hours=72))
                            tm_event = tm_int[boolarr]
                            creep_event = creep_int[boolarr]
                            
                                                


                            fig=plt.figure(figsize=(20,10))
                            fig.tight_layout()
                            ax = plt.subplot(1,1,1)
                            if len(creep_event) >= 1:
                                plt.plot(tm_event,creep_event-creep_event[0],marker='o', color='b', linestyle='-', linewidth=2, markersize=6)
                                bottom,top = ax.get_ylim()
                                plt.vlines(x=pd.to_datetime(df_picks.ST.iloc[j]),ymin=bottom,ymax=top)
                                if j < len(df_picks)-1:
                                    plt.vlines(x=pd.to_datetime(df_picks.ST.iloc[j+1]),ymin=bottom,ymax=top)
                                ax.set_ylim(bottom,top)   
                                ax.set_xlim(tm_event[0],tm_event[-1])
                                scale = 1.1
                                zp = cep.ZoomPan()
                                figZoom = zp.zoom_factory(ax, base_scale = scale)
                                figPan = zp.pan_factory(ax)
                                s = 1
                                plt.title('{k}/{p}'.format(k=j+1,p=len(df_picks)))
                                plt.show()
                                pts = plt.ginput(1,timeout = -1)
                                plt.close()
                                for sublist in pts:
                                    df = pd.DataFrame({'ET':[sublist[0]]})
                                    df_picks_end = pd.concat([df_picks_end,df]) 
                                    #print(df_picks_end)
                            else:
                                df = pd.DataFrame({'ET':[dt.datetime(2100,1,1,0,0,0)]})
                                df_picks_end = pd.concat([df_picks_end,df])

                        for r in range(len(df_picks)):
                            time_convert = mdates.num2date(float(df_picks_end.ET.iloc[r]))
                            s3 = pd.Series(time_convert).dt.round("1min")
                            df_picks_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)  
                        
                        #df_picks_end.ET= mdates.num2date(df_picks_end.ET)
                        df_picks_end.reset_index(inplace=True)
                        df_picks_end.drop(columns=['index'],inplace=True)

                        

                        df_picks['ET'] = np.ones(len(df_picks))
                        for r in range(len(df_picks)):
                            df_picks.ET.iloc[r]=df_picks_end.ET.iloc[r]

                        df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =creepmeter_info.File_code.iloc[i]))

Key: BAL1_1.0mins
extracting data
['Slip_BAL1_1.0mins', 'Time_BAL1_1.0mins']
Load in creepmeter data from a SAC file
scaling data if from Roger
Picking
10


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_6654/2494705723.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_picks.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_6654/2494705723.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_picks.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_6654/2494705723.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

IndexError: single positional indexer is out-of-bounds

In [9]:
df_picks

,Unnamed: 0,ST,ET,ET_new,ST_new,Quality_check_new,Quality_check_final,ET_final,ST_final
0,0.0,2023-03-12 10:47:00,2023-03-13 18:56:00,2023-03-13 21:37:00,2023-03-12 10:47:00,CE,NCE,2023-03-13 18:55:55,2023-03-12 10:47:55
1,1.0,2023-03-14 09:22:00,2023-03-15 08:15:00,2023-03-15 12:59:00,2023-03-14 07:47:00,CE,CE,2023-03-15 08:14:55,2023-03-14 07:39:55
2,2.0,2023-03-20 10:00:00,2023-03-21 08:26:00,2023-03-21 08:26:00,2023-03-20 10:00:00,CE,CE,2023-03-21 08:25:55,2023-03-20 09:39:55
0,NaN,2023-08-22 15:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,2023-08-23 03:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,2023-08-23 07:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,2023-08-23 12:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,2023-08-23 21:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,2023-08-23 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,2023-08-24 05:33:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =creepmeter_info.File_code.iloc[i]))

In [9]:
tr.data

array([-31.75659943, -31.77729988, -31.77729988, ...,   9.60445023,
         9.60204029,   9.60445023])

In [9]:
path = "../../Data/DATA_tidied/SAC/SC30_1T.SAC"

st = obspy.read(path)
tr = st[0]
tr.interpolate(sampling_rate=tr.stats.sampling_rate)
tm = tr.times("utcdatetime")
creep = tr.data
sample_rate =  round(tr.stats.delta)/60
tm_dt = []
for w in range(len(tm)):
    tm_new = tm[w].datetime
    tm_dt.append(tm_new)

print('Interpolate')
tm_int, creep_int, upsampled = cep.interpolate(tm_dt,creep,sample_rate)
plt.figure()
plt.plot(tm_dt,creep)
plt.show()

Interpolate


In [25]:
tm_diff = np.diff(tm_dt)

In [29]:
plt.figure()
plt.plot(tm_diff/dt.timedelta(minutes=1))

In [27]:
tm_diff

array([datetime.timedelta(seconds=299, microseconds=999993),
       datetime.timedelta(seconds=299, microseconds=999992),
       datetime.timedelta(seconds=299, microseconds=999993), ...,
       datetime.timedelta(seconds=299, microseconds=999993),
       datetime.timedelta(seconds=299, microseconds=999993),
       datetime.timedelta(seconds=299, microseconds=999992)], dtype=object)